In [1]:
from dataset import SpectrogramTripletDataset
import numpy as np
import torch

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
data = np.load('spectrograms.npy', allow_pickle=True)
data = torch.tensor(data, dtype=torch.float32).to(device)
labels = np.load('classes.npy', allow_pickle=True)
labels = torch.tensor(labels, dtype=torch.long).to(device)
train_dataset = SpectrogramTripletDataset(data, labels)
val_dataset = train_dataset.give_val_set(0.2)

batch_size = 32
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=batch_size, shuffle=True)

batch = next(iter(train_loader))
print(batch[0].dtype)

# idx = torch.randint(0, len(train_dataset), (1,)).item()
# im1, im2, im3 = train_dataset[idx]
# print(im1.shape, im2.shape, im3.shape)

c:\Users\joeag\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


torch.float32


In [2]:
labels = torch.randint(0, 10, (100,))
possible_idxs = torch.where(labels == 7)[0]
same_idx = possible_idxs[torch.randint(0, len(possible_idxs), (1,)).item()]
same_idx

tensor(97)

In [3]:
x = torch.randn(10, 5, 50, 50)
x[0].shape

torch.Size([5, 50, 50])

In [6]:
import torch
import torch.nn as nn
from model import Transformer
import numpy as np
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model = Transformer(256, 128).to(device)
x = torch.randn(32, 5, 50, 50).to(device)
y = torch.randn(32, 5, 50, 50).to(device)
z = torch.randn(32, 5, 50, 50).to(device)
model(x).shape, model(y).shape, model(z).shape

embedded inp shape: torch.Size([32, 50, 256])
embedded inp shape: torch.Size([32, 50, 256])
embedded inp shape: torch.Size([32, 50, 256])


(torch.Size([32, 128]), torch.Size([32, 128]), torch.Size([32, 128]))

In [11]:
data = np.load('spectrograms.npy', allow_pickle=True)
data = torch.tensor(data, dtype=torch.float32).to(device)
labels = np.load('classes.npy', allow_pickle=True)
labels = torch.tensor(labels, dtype=torch.long).to(device)
config = {
    'num_features': 128,
    'split_ratio': 0.2
}
train_dataset = SpectrogramTripletDataset(data, labels)
val_dataset = train_dataset.give_val_set(config['split_ratio'])

# model = CNN(config['num_features']).to(device)
model = Transformer(config['num_features']*2, config['num_features']).to(device)

trainloader = torch.utils.data.DataLoader(train_dataset, batch_size=32, shuffle=True)
valloader = torch.utils.data.DataLoader(val_dataset, batch_size=32, shuffle=True)

for batch in trainloader:
    x, y, z = batch
    x = model(x)
    print(x.shape, y.shape, z.shape)

embedded inp shape: torch.Size([32, 50, 256])
torch.Size([32, 128]) torch.Size([32, 5, 50, 50]) torch.Size([32, 5, 50, 50])
embedded inp shape: torch.Size([32, 50, 256])
torch.Size([32, 128]) torch.Size([32, 5, 50, 50]) torch.Size([32, 5, 50, 50])
embedded inp shape: torch.Size([32, 50, 256])
torch.Size([32, 128]) torch.Size([32, 5, 50, 50]) torch.Size([32, 5, 50, 50])
embedded inp shape: torch.Size([32, 50, 256])
torch.Size([32, 128]) torch.Size([32, 5, 50, 50]) torch.Size([32, 5, 50, 50])
embedded inp shape: torch.Size([32, 50, 256])
torch.Size([32, 128]) torch.Size([32, 5, 50, 50]) torch.Size([32, 5, 50, 50])
embedded inp shape: torch.Size([32, 50, 256])
torch.Size([32, 128]) torch.Size([32, 5, 50, 50]) torch.Size([32, 5, 50, 50])
embedded inp shape: torch.Size([32, 50, 256])
torch.Size([32, 128]) torch.Size([32, 5, 50, 50]) torch.Size([32, 5, 50, 50])
embedded inp shape: torch.Size([32, 50, 256])
torch.Size([32, 128]) torch.Size([32, 5, 50, 50]) torch.Size([32, 5, 50, 50])
embedded

KeyboardInterrupt: 

In [3]:
import torch
import torch.nn as nn
layer = nn.Conv2d(5, 128, (50, 5), 1, 0)
x = torch.randn(32, 5, 50, 50)
layer(x).shape

torch.Size([32, 128, 1, 46])

In [15]:
device= torch.device('cuda' if torch.cuda.is_available() else 'cpu')
class ConvBlock(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size, stride, padding):
        super(ConvBlock, self).__init__()
        self.conv = nn.Conv2d(in_channels, out_channels, kernel_size=kernel_size, stride=stride, padding=padding)
        self.bn = nn.BatchNorm2d(out_channels)
        self.relu = nn.ReLU()

    def forward(self, x):
        return self.relu(self.bn(self.conv(x)))

class CNN(nn.Module):
    def __init__(self, num_features=8, out_channels=128):
        super().__init__()
        self.conv_blocks = nn.ModuleList([
            ConvBlock(5, num_features, (50, 5), 5, 0),
            ConvBlock(num_features, num_features*2, (1,3), 1, 0),
            ConvBlock(num_features*2, num_features*4, (1,3), 1, 0),
            ConvBlock(num_features*4, num_features*4, (1,3), 1, 0),
            ConvBlock(num_features*4, num_features*4, (1,4), 1, 0),
        ])

        self.lin_layers = nn.Sequential(
            nn.Linear(num_features*4, out_channels),
            nn.ReLU(),
            nn.Linear(out_channels, out_channels),
        )

    def forward(self, x):
        for block in self.conv_blocks:
            x = block(x)
        x = x.flatten(1)
        x = self.lin_layers(x)
        return x

model = CNN(16, 128).to(device)
x = torch.randn(32, 5, 50, 50).to(device)
model(x).shape

torch.Size([32, 128])

In [16]:
x = torch.randn(32, 5, 50, 50).to(device)
model(x).shape

torch.Size([32, 128])